In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set Matplotlib style for better visualization
plt.style.use('ggplot')

In [ ]:
# Load the Sentiment Data
sentiment_df = pd.read_csv("fear_greed_index.csv")

# Convert the 'date' column to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df = sentiment_df.set_index('date')

# Drop the redundant 'timestamp' (Unix) column
sentiment_df = sentiment_df.drop(columns=['timestamp'])

# Rename columns for clarity in the final merged table
sentiment_df = sentiment_df.rename(columns={
    'value': 'Sentiment_Value',
    'classification': 'Sentiment_Classification'
})

print("Sentiment Data Prepared (Head):")
print(sentiment_df.head())

Sentiment Data Prepared (Head):
            Sentiment_Value Sentiment_Classification
date                                                
2018-02-01               30                     Fear
2018-02-02               15             Extreme Fear
2018-02-03               40                     Fear
2018-02-04               24             Extreme Fear
2018-02-05               11             Extreme Fear


In [ ]:
# Load the Trader Data
trader_df = pd.read_csv("historical_data.csv")

# Convert 'Timestamp IST' to datetime object (using DD-MM-YYYY format)
# Note: This is crucial for correct date parsing
trader_df['Timestamp IST'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')

# Extract the date part to create a daily trade date column
trader_df['Trade_Date'] = trader_df['Timestamp IST'].dt.date

# Aggregate performance metrics by day:
# 1. Daily_PnL: Total daily profitability
# 2. Daily_Volume_USD: Total daily traded volume
daily_performance_df = trader_df.groupby('Trade_Date').agg(
    Daily_PnL=('Closed PnL', 'sum'),
    Daily_Volume_USD=('Size USD', 'sum')
)

# Convert the index back to datetime object for merging with sentiment_df
daily_performance_df.index = pd.to_datetime(daily_performance_df.index)
daily_performance_df.index.name = 'Date'

print("\nTrader Data Aggregated (Head):")
print(daily_performance_df.head())


Trader Data Aggregated (Head):
             Daily_PnL  Daily_Volume_USD
Date                                    
2023-05-01    0.000000            477.00
2023-12-05    0.000000          50005.83
2023-12-14 -205.434737         113203.35
2023-12-15  -24.632034          10609.95
2023-12-16    0.000000          15348.77


In [ ]:
# Perform the Inner Merge on the Date index
merged_df = daily_performance_df.merge(sentiment_df,
                                       left_index=True,
                                       right_index=True,
                                       how='inner')

print("\nMerged Data (Sentiment + Performance) Info:")
merged_df.info()

print("\nMerged Data Head:")
print(merged_df.head())


Merged Data (Sentiment + Performance) Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 479 entries, 2023-05-01 to 2025-05-01
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily_PnL                 479 non-null    float64
 1   Daily_Volume_USD          479 non-null    float64
 2   Sentiment_Value           479 non-null    int64  
 3   Sentiment_Classification  479 non-null    object 
dtypes: float64(2), int64(1), object(1)
memory usage: 18.7+ KB

Merged Data Head:
             Daily_PnL  Daily_Volume_USD  Sentiment_Value  \
2023-05-01    0.000000            477.00               63   
2023-12-05    0.000000          50005.83               75   
2023-12-14 -205.434737         113203.35               72   
2023-12-15  -24.632034          10609.95               70   
2023-12-16    0.000000          15348.77               67   

           Sentiment_Classification  
2023-05-01     

In [ ]:
# 1. Group and calculate average daily PnL
profitability_by_sentiment = merged_df.groupby('Sentiment_Classification')['Daily_PnL'].mean().sort_values(ascending=False)

# Function to assign colors based on sentiment (Fear/Greed)
def get_colors(index):
    colors = []
    for sentiment in index:
        if 'Fear' in sentiment:
            colors.append('skyblue')
        elif 'Greed' in sentiment:
            colors.append('lightcoral')
        else:
            colors.append('lightgray')
    return colors

colors = get_colors(profitability_by_sentiment.index)

# 2. Plotting the results
plt.figure(figsize=(10, 6))
bars = plt.bar(profitability_by_sentiment.index, profitability_by_sentiment.values, color=colors)

# Add labels and horizontal line
for bar in bars:
    yval = bar.get_height()
    label = f'${yval:,.0f}'
    plt.text(bar.get_x() + bar.get_width()/2, yval, label, ha='center', va='bottom' if yval > 0 else 'top', fontsize=9)

plt.axhline(0, color='black', linewidth=0.8)
plt.title('Average Daily PnL by Market Sentiment Classification')
plt.xlabel('Sentiment Classification')
plt.ylabel('Average Daily Net PnL (USD)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the chart to the 'outputs/' folder (as required by instructions)
plt.savefig('profitability_by_sentiment.png')
plt.close()
print("\nChart 1: 'profitability_by_sentiment.png' created.")


Chart 1: 'profitability_by_sentiment.png' created.


In [ ]:
# 1. Group and calculate average daily Volume
volume_by_sentiment = merged_df.groupby('Sentiment_Classification')['Daily_Volume_USD'].mean().sort_values(ascending=False)

colors = get_colors(volume_by_sentiment.index) # Re-use the color function

# 2. Plotting the results
plt.figure(figsize=(10, 6))
bars = plt.bar(volume_by_sentiment.index, volume_by_sentiment.values, color=colors)

# Add labels
for bar in bars:
    yval = bar.get_height()
    # Format the Volume value for the label (in thousands)
    label = f'${yval/1000:,.0f}K'
    plt.text(bar.get_x() + bar.get_width()/2, yval, label, ha='center', va='bottom', fontsize=9)

plt.title('Average Daily Trading Volume (USD) by Market Sentiment Classification')
plt.xlabel('Sentiment Classification')
plt.ylabel('Average Daily Volume (USD)')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.gca().ticklabel_format(axis='y', style='plain', useOffset=False) # Prevent scientific notation on y-axis
plt.tight_layout()

# Save the chart to the 'outputs/' folder (as required by instructions)
plt.savefig('volume_by_sentiment.png')
plt.close()
print("Chart 2: 'volume_by_sentiment.png' created.")

Chart 2: 'volume_by_sentiment.png' created.
